#  Методы оценки АВ тестов 

Предисловие:
1. Для каждого теста содержится код с необходимыми библиотеками, чтобы можно было запускать автономно.
2. Можно и нужно импортировать свои данные, для этого либо заведены объекты df, либо переменные
3.  Во всех тестах нулевая и альтернативная гипотезы:


##### H0: Между группами нет разницы
##### H1: Между группами есть статистически значимая разница

<b><font size=4>Содержание</font></b><a name="to_content1."></a>

- **[1. Дискретные метрики](#h1)**
  - [1.1 Тест Фишера](#h1.1)
  - [1.2 Хи-квадрат(тест Пирсона)](#h1.2)
- **[2. Непрерывные метрики](#h2)**
  - [2.1 Тест Стьюдента (t-test)](#h2.1)
  - [2.2 Тест Уэлча (Welch's test)](#h2.2)
  - [2.3 U-тест Манна-Уитни (Mann-Whitney test)](#h2.3)
- **[3. Ratio-метрики, метрики соотношения](#h3)**
  - [3.1 Бустреп (bootstrap)](#h3.1)
  - [3.2 Дельта-метод](#h3.2)
  - [3.2 CUPED-метод](#h3.3)
- **[4. Множественное тестирование](#h4)**
  - [4.1 Поправка Бонферонни](#h4.1)
  - [4.2 Метод Холма](#h4.2)

# 1. Дискретные 
(качественные или метрики - пропорции, такие как конверсия, CR)<a class="anchor" id="h1"></a>


## 1.1.Тест Фишера<a class="anchor" id="h1.1"></a>
[<font size="2">(к содержанию)</font>](#to_content1.)

Предпосылки:
1. для выборок маленьких размеров

In [1]:
import numpy as np
import scipy.stats as stats

In [2]:
pvalue_fisher_test = 0.05 #уровень надежности

#Данные по группам

fisher_a_denum = 32655 #знаменатель в группе А
fisher_a_num = 129 #числитель в группе А
fisher_b_denum = 32855 #знаменатель в группе B
fisher_b_num = 186 #числитель в группе B


group_A = [fisher_a_num, fisher_a_denum]
group_B = [fisher_b_num, fisher_b_denum]

#Расчет коэфф-та
table = [group_A, group_B]
odd_ratio, p_value = stats.fisher_exact(table)

print("- Fisher's t-test:")
# Результат
print('Значение odd ratio: ' + str(odd_ratio))
print('Значение p_value : ' + str(p_value))

# Проверяем есть ли изменение
if p_value < pvalue_fisher_test:
    print("Невозможно подтвердить гипотезу об отсутствии разницы между группами. На заданном уровне надежности может наблюдаться разница между группами")
else:
    print("Невозможно опровергнуть гипотезу об отсутствии разницы между группами")

- Fisher's t-test:
Значение odd ratio: 0.6977961187586745
Значение p_value : 0.0018504812506557238
Невозможно подтвердить гипотезу об отсутствии разницы между группами. На заданном уровне надежности может наблюдаться разница между группами


## 1.2. Хи-квадрат (тест Пирсона)<a class="anchor" id="h1.2"></a>
[<font size="2">(к содержанию)</font>](#to_content1.)

Предпосылки:
1. тест для дискретных метрик


In [3]:
import numpy as np
import scipy.stats as stats

In [4]:
p_value_chi = 0.05 #уровень надежности

# данные по группам
# в формате group_A = [кол-во успешных событий, общее кол-во событий]
pearson_a = [129, 32655]
pearson_b = [186, 32855]


# Проверка на Sample Ratio Mismatch
if abs(1-pearson_a[1]/pearson_b[1])>= 0.05:
    print ('SRM alert! Выборки неравномерные')

# Оценка статистики
chi2, p, dof, ex = stats.chi2_contingency([pearson_a, pearson_b], correction=False)

# Рассчитываем доверительный интервал для изменения
lift = (pearson_b[0]/pearson_b[1])/(pearson_a[0]/pearson_a[1])
std_error = np.sqrt(1/pearson_b[0] + 1/pearson_b[1] + 1/pearson_a[0] + 1/pearson_a[1])
ci = stats.norm.interval(1-p_value_chi, loc=lift, scale=std_error)

# Выводим результаты
print("- Pearson's t-test:")
print("p-value: ", p)
print("Доверительный интервал изменения: ", ci)

# Проверяем есть ли изменение
if p < p_value_chi and ci[0] > 1:
    print("Невозможно подтвердить гипотезу об отсутствии разницы между группами. Найденое различие на заданном уровне надежности может наблюдаться")
else:
    print("Невозможно опровергнуть гипотезу об отсутствии разницы между группами")

- Pearson's t-test:
p-value:  0.0016334785152705034
Доверительный интервал изменения:  (1.2079915600322153, 1.6581751414361816)
Невозможно подтвердить гипотезу об отсутствии разницы между группами. Найденое различие на заданном уровне надежности может наблюдаться


In [5]:
# Вариант "из коробки"
import scipy
from scipy import stats

p_value_chi = 0.05 #уровень надежности

pearson_a = [129, 32655]
pearson_b = [186, 32855]

p = scipy.stats.chi2_contingency([pearson_a, pearson_b],correction=False)[1]
ci = stats.norm.interval(1-p_value_chi, loc=lift, scale=std_error)
print("- Pearson's t-test:")
print("p-value: ", p)

# Проверяем есть ли изменение
if p < p_value_chi and ci[0] > 1:
    print("Невозможно подтвердить гипотезу об отсутствии разницы между группами. Найденое различие на заданном уровне надежности может наблюдаться")
else:
    print("Невозможно опровергнуть гипотезу об отсутствии разницы между группами")

- Pearson's t-test:
p-value:  0.0016334785152705034
Невозможно подтвердить гипотезу об отсутствии разницы между группами. Найденое различие на заданном уровне надежности может наблюдаться


# 2. Непрерывные
(количественные -  средняя выручка на пользователя, средняя длина сессии)<a class="anchor" id="h2"></a>

[<font size="2">(к содержанию)</font>](#to_content1.)

## 2.1. T-test Стьюдента<a class="anchor" id="h2.1"></a>
[<font size="2">(к содержанию)</font>](#to_content1.)

Предпосылки:
1. Независимость выборок
2. Равная дисперсия выборок (приблизительно)


In [6]:
import numpy as np
from scipy.stats import chi2_contingency, mannwhitneyu, norm, t, ttest_ind
import pandas as pd

In [7]:
#Тут прописано ручками, как считается тест(чтобы было понятно)

p_value_t = 0.05 #уровень надежности

np.random.seed(42)
n_a, n_b = 17, 14 #длина выборок

d1 = norm(loc=200, scale=100)
d2 = norm(loc=280, scale=90)
disc = 50
student_a = (d1.rvs(size=n_a) / disc).astype(int) * disc
student_b = (d2.rvs(size=n_b) / disc).astype(int) * disc

# импорт своих значений
# student_a = pd.read_csv()
# student_b = pd.read_csv()
# n_a  = len(student_a)
# n_b  = len(student_b)


# дисперсия
s_student_a = np.sqrt(np.var(student_a, ddof=1))
s_student_b= np.sqrt(np.var(student_b, ddof=1))

# Pooled variance
s_p = np.sqrt(((n_a-1) * s_student_a**2 +
               (n_b-1) * s_student_b**2)
              / (n_a + n_b - 2))

# Test statistic distribution under null hypothesis H0
dofs = n_a + n_b - 2
stat_distrib = t(df=dofs, loc=0, scale=1)

# t value
t_val = (student_a.mean() - student_b.mean()) / (s_p * np.sqrt(1/n_a + 1/n_b))

# p-value
p_t_test = stat_distrib.cdf(t_val) * 2

print("- Student's t-test:")
print(f"   - Значение t статистики: {t_val:.3f}")
print(f"   - Значение p-value: {p_t_test*100:.7f}%")

# Проверяем есть ли изменение
if p_t_test < p_value_t:
    print("Невозможно подтвердить гипотезу об отсутствии разницы между группами. Найденое различие на заданном уровне надежности может наблюдаться")
else:
    print("Невозможно опровергнуть гипотезу об отсутствии разницы между группами")

- Student's t-test:
   - Значение t статистики: -1.789
   - Значение p-value: 8.4055453%
Невозможно опровергнуть гипотезу об отсутствии разницы между группами


In [8]:
# Вариант "из коробки"
import scipy
from scipy.stats import chi2_contingency, mannwhitneyu, norm, t, ttest_ind
import numpy as np
import pandas as pd

In [9]:
p_t_test = 0.05 #уровень надежности

np.random.seed(42)
n_a, n_b = 17, 14 #длина выборок

d1 = norm(loc=200, scale=100)
d2 = norm(loc=280, scale=90)
disc = 50
student_a = (d1.rvs(size=n_a) / disc).astype(int) * disc
student_b = (d2.rvs(size=n_b) / disc).astype(int) * disc

# импорт своих значений
# student_a = pd.read_csv()
# student_b = pd.read_csv()
# n_a  = len(student_a)
# n_b  = len(student_b)


t_val, p_val = stats.ttest_ind(student_a, student_b)

print("- Student's t-test:")
print(f"   - Значение t статистики: {t_val:.3f}")
print(f"   - Значение p-value: {p_val*100:.7f}%")


# Проверяем есть ли изменение
if p_val < p_t_test:
    print("Невозможно подтвердить гипотезу об отсутствии разницы между группами. Найденое различие на заданном уровне надежности может наблюдаться")
else:
    print("Невозможно опровергнуть гипотезу об отсутствии разницы между группами")

- Student's t-test:
   - Значение t статистики: -1.789
   - Значение p-value: 8.4055453%
Невозможно опровергнуть гипотезу об отсутствии разницы между группами


## 2.2. Welch t-test (t-test Уэлча)<a class="anchor" id="h2.2"></a>
[<font size="2">(к содержанию)</font>](#to_content1.)

Предпосылки:
1. Независимость выборок


In [10]:
# Вариант "из коробки"
#Тот же t-тест, с поправкой на неравную дисперсию
import scipy
from scipy import stats
import pandas as pd

In [11]:
p_welch_test = 0.05 #уровень надежности
np.random.seed(42)
n_a, n_b = 17, 14 #длина выборок

d1 = norm(loc=200, scale=100)
d2 = norm(loc=280, scale=90)
disc = 50
welch_a = (d1.rvs(size=n_a) / disc).astype(int) * disc
welch_b = (d2.rvs(size=n_b) / disc).astype(int) * disc

# импорт своих значений
# welch_a = pd.read_csv()
# welch_b = pd.read_csv()
# n_a  = len(student_a)
# n_b  = len(student_b)

t_val, p_val = stats.ttest_ind(welch_a, welch_b, equal_var=False)

print("- Welch's t-test:")
print(f"   - Значение t статистики: {t_val:.3f}")
print(f"   - Значение p-value: {p_val*100:.7f}%")


# Проверяем есть ли изменение
if p_val < p_welch_test:
    print("Невозможно подтвердить гипотезу об отсутствии разницы между группами. Найденое различие на заданном уровне надежности может наблюдаться")
else:
    print("Невозможно опровергнуть гипотезу об отсутствии разницы между группами")


- Welch's t-test:
   - Значение t статистики: -1.848
   - Значение p-value: 7.5003236%
Невозможно опровергнуть гипотезу об отсутствии разницы между группами


## 2.3. U-test Манна-Уитни<a class="anchor" id="h2.3"></a>
[<font size="2">(к содержанию)</font>](#to_content1.)

Предпосылки:
1. Маленькая выборка
2. Распределение скошено: в результате теста произошло масштабирование или сдвиг выборки

In [12]:
# Вариант "из коробки"
import scipy
from scipy import stats

In [13]:
p_mu_test = 0.05 #уровень надежности
np.random.seed(42)
n_a, n_b = 17, 14 #длина выборок

d1 = norm(loc=200, scale=100)
d2 = norm(loc=280, scale=90)
disc = 50
mu_a = (d1.rvs(size=n_a) / disc).astype(int) * disc
mu_b = (d2.rvs(size=n_b) / disc).astype(int) * disc

# импорт своих значений
# welch_a = pd.read_csv()
# welch_b = pd.read_csv()
# n_a  = len(student_a)
# n_b  = len(student_b)

u_val, p_val = stats.mannwhitneyu(mu_a, mu_b, alternative='two-sided')

print("- Mann-Whitney U-test:")
print(f"   - Значение U статистики: {u_val:.3f}")
print(f"   - Значение p-value: {p_val*100:.7f}%")


# Проверяем есть ли изменение
if p_val < p_mu_test:
    print("Невозможно подтвердить гипотезу об отсутствии разницы между группами. Найденое различие на заданном уровне надежности может наблюдаться")
else:
    print("Невозможно опровергнуть гипотезу об отсутствии разницы между группами")


- Mann-Whitney U-test:
   - Значение U статистики: 76.000
   - Значение p-value: 8.3613250%
Невозможно опровергнуть гипотезу об отсутствии разницы между группами


# 3. Ratio метрики, метрики соотношения
(количественные - средний чек, CTR)<a class="anchor" id="h3"></a>

[<font size="2">(к содержанию)</font>](#to_content1.)


## 3.1. Бутстреп (bootstrap)<a class="anchor" id="h3.1"></a>
[<font size="2">(к содержанию)</font>](#to_content1.)

Предпосылки:
1. Обращаем внимание, на разницу каких метрики мы тестируем (среднее,квантиль и т.д)
2. Основная особенность для метрик отношения — данные необходимо семплировать по объектам, а не по наблюдениям. Если объект оказался в определённой группе, то и все его действия будут в этой группе. Для этого нужна [предвариательная обработка](#h4.1): расчет полной стоимости покупок по юзеру (например).

In [14]:
import scipy
from scipy import stats
import numpy as np
import pandas as pd

In [15]:
n = 1000 #размер выборки
B = 10000 #кол-во итераций
p_val_boostrap = 0.05 #уровень надежности

#генерация выборок
boostrap_a = np.random.normal(90, 20, n)
boostrap_b = np.random.normal(90, 15, n)

# импорт своих значений
# boostrap_a = pd.read_csv()
# boostrap_b = pd.read_csv()

# длина выборок
len_a = len(boostrap_a)
len_b = len(boostrap_b)

# считаем разницу метрик отношения (если смотрим не на среднее
# , а на квантиль, медиану и тд, заменить в формуле)
bootstrap_mean_a = []
bootstrap_mean_b = []
bootstrap_stats = []

# генерим подвыборки
for _ in range(B):
    bootstrap_values_a = np.random.choice(boostrap_a, len_a, True)
    bootstrap_mean_a.append(np.mean(bootstrap_values_a))
    bootstrap_values_b = np.random.choice(boostrap_b, len_b, True)
    bootstrap_mean_b.append(np.mean(bootstrap_values_b))
    
# записываем значения разницы метрик
for i, j in zip(bootstrap_mean_a, bootstrap_mean_b):
    bootstrap_stats_value = i-j
    bootstrap_stats.append(bootstrap_stats_value)

# строим доверительный интервал для статистики
# проверить распределение статистики и построить доверит интервал для нее!!!
# ТОЧНО ЛИ ОН ТАК СТРОИТСЯ????
ci = np.percentile(bootstrap_stats, [p_val_boostrap*100/2, 100-p_val_boostrap*100/2])
# проводим т-тест, проверяем,попадаем ли ноль в этот доверительны интервал
t_val, p_val_boostrap_test = stats.ttest_1samp(bootstrap_stats, 0)

print("- Bootstrap test:")
print("Bootstrap mean difference 95% CI: ({:.2f}, {:.2f})".format(ci[0], ci[1]))
print("t-statistic: ", t_val)
print(f"p-value: {p_val_boostrap_test:0.7f}")

# Проверяем есть ли изменение
if p_val_boostrap_test < p_val_boostrap:
    print("Невозможно подтвердить гипотезу об отсутствии разницы между группами. Найденое различие на заданном уровне надежности может наблюдаться")
else:
    print("Невозможно опровергнуть гипотезу об отсутствии разницы между группами. На заданном уровне надежности не найдено статистически занчимое различие между группами.")


- Bootstrap test:
Bootstrap mean difference 95% CI: (-1.51, 1.56)
t-statistic:  1.3774519814981743
p-value: 0.1684034
Невозможно опровергнуть гипотезу об отсутствии разницы между группами. На заданном уровне надежности не найдено статистически занчимое различие между группами.


#### Предварительная обработка<a class="anchor" id="h4.1"></a>
!раздел в разработке!

In [16]:
# def generate_data(sample_size, effect):
#     """Генерирует данные со стоимостью покупок.

#     Возвращает два списка с данными контрольной и экспериментальной групп.
#     Элементы списков - множества со стоимостями покупок пользователей. 
#     """
#     result = []
#     for group_effect in [0, effect]:
#         n_purchases = np.random.randint(1, 5, sample_size)
#         mean_costs = np.random.uniform(1000, 2000, sample_size)
#         data = [
#             np.random.normal(mean + group_effect, 200, n)
#             for n, mean in zip(n_purchases, mean_costs)
#         ]
#         result.append(data)
#     return result


In [17]:
# def get_percentile_ci(bootstrap_stats, alpha):
#     """Строит перцентильный доверительный интервал."""
#     left, right = np.quantile(bootstrap_stats, [alpha / 2, 1 - alpha / 2])
#     return left, right

# def check_bootsrtap(a, b, n_iter, alpha):
#     """Оценивает значимость отличий с помощью бутстрепа.

#     Если отличия значимые, то возвращает 1, иначе 0.
#     """
#     # вычисляем стоимость и количество покупок клиентов
#     xy_a = np.array([[sum(values), len(values)] for values in a])
#     xy_b = np.array([[sum(values), len(values)] for values in b])
#     # генерируем случайные индексы для выбора подмножеств данных
#     len_a = len(a)
#     len_b = len(b)
#     indexes_a = np.random.choice(
#         np.arange(len_a), size=(n_iter, len_a), replace=True
#     )
#     indexes_b = np.random.choice(
#         np.arange(len_b), size=(n_iter, len_b), replace=True
#     )

#     bootstrap_stats = []
#     for idx_a, idx_b in zip(indexes_a, indexes_b):
#         bootstrap_xy_a = xy_a[idx_a]
#         bootstrap_xy_b = xy_b[idx_b]
#         # считаем разницу метрик отношения
#         bootstrap_stat = (
#             bootstrap_xy_b[:, 0].sum() / bootstrap_xy_b[:, 1].sum()
#             - bootstrap_xy_a[:, 0].sum() / bootstrap_xy_a[:, 1].sum()
#         )
#         bootstrap_stats.append(bootstrap_stat)
#     # строим доверительный интервал и оцениваем значимость отличий
#     ci = get_percentile_ci(bootstrap_stats, alpha)
#     has_effect = 1 - (ci[0] < 0 < ci[1])
#     return has_effect

# alpha = 0.05               # допустимая вероятность ошибки I рода
# sample_size = 1000         # размер групп
# n_iter = 1000              # количество итераций бутстрепа

# effects = []
# for _ in range(1000):
#     a, b = generate_data(sample_size, 0)
#     has_effect = check_bootsrtap(a, b, n_iter, alpha)
#     effects.append(has_effect)

# error_rate = np.mean(np.array(effects) == 1)
# print(f'Доля ошибок первого рода: {error_rate:0.3f}')

## 3.2. Дельта-метод<a class="anchor" id="h3.2"></a>
[<font size="2">(к содержанию)</font>](#to_content1.)
Предпосылки:
1. Зависимые данные


In [18]:
import numpy as np
import pandas as pd
import scipy
from scipy import stats

In [19]:
n = 1000 #размер выборки
p_val_delta = 0.05 #уровень надежности

#генерация выборок
num_a = np.random.normal(90, 20, n) #числитель метрики отношения группы А
denom_a = np.random.normal(90, 20, n) #знаменатель метрики отношения группы А

num_b = np.random.normal(90, 15, n) #числитель метрики отношения группы B
denom_b = np.random.normal(90, 15, n) #знаменатель метрики отношения группы B

# импорт своих значений
# numer_a = pd.read_csv()
# denom_a = pd.read_csv()
# numer_a = pd.read_csv()
# denom_a = pd.read_csv()

In [20]:
#расчет метрик для группы А
n_user_a = len(denom_b)

# оценка ctr группы а
ctr_a = sum(num_a)/sum(denom_a)

# оценка дисперсии
mean_x, mean_y = np.mean(num_a), np.mean(denom_a)
var_x, var_y = np.var(num_a, ddof = 1), np.var(denom_a, ddof = 1)
cov_xy = np.cov(num_a, denom_a, ddof = 1)[0, 1]

# оценка дисперсии метрики
var_metric_a = (
            var_x / mean_y ** 2
            - 2 * (mean_x / mean_y ** 3) * cov_xy
            + (mean_x ** 2 / mean_y ** 4) * var_y
        ) / n_user_a

In [21]:
#расчет метрик для группы B
n_user_b = len(denom_b)

# оценка ctr группы b
ctr_b = sum(num_b)/sum(denom_b)

# оценка дисперсии
mean_n, mean_m = np.mean(num_b), np.mean(denom_b)
var_n, var_m = np.var(num_b, ddof = 1), np.var(denom_b, ddof = 1)
cov_nm = np.cov(num_b, denom_b, ddof = 1)[0, 1]

# оценка дисперсии метрики
var_metric_b = (
            var_n / mean_m ** 2
            - 2 * (mean_n / mean_m ** 3) * cov_nm
            + (mean_n ** 2 / mean_m ** 4) * var_m
        ) / n_user_b

In [22]:
#t-test
def ttest(mean_control,mean_treatment,var_control,var_treatment):
    diff = mean_treatment - mean_control
    var = var_control+var_treatment
    stde = 1.96*np.sqrt(var)
    lower = diff - stde 
    upper = diff + stde
    t_val = diff/np.sqrt(var)
    p_val_delta_test = stats.norm.sf(abs(t_val))*2

#     result = {'mean_control':ctr_a,
#              'mean_experiment':ctr_b,
#              'var_control':var_metric_a,
#              'var_experiment':var_metric_b,
#              'difference':diff,
#              'lower_bound':lower,
#              'upper_bound':upper,
#               't_val': t_val,
#              'p-value': p_val_delta_test}
    result = [ctr_a,ctr_b,var_metric_a,var_metric_b,diff,lower,upper, t_val, p_val_delta_test]
    return result

t_val = ttest(ctr_a,ctr_b, var_metric_a,var_metric_b)[7]
p_val_delta_test = ttest(ctr_a,ctr_b, var_metric_a,var_metric_b)[8]

print("- Delta t-test:")
print("Значение T статистики: ", t_val)
print(f"p-value: {p_val_delta_test:0.7f}")

# Проверяем есть ли изменение
if p_val_delta_test < p_val_delta:
    print("Невозможно подтвердить гипотезу об отсутствии разницы между группами. Найденое различие на заданном уровне надежности может наблюдаться")
else:
    print("Невозможно опровергнуть гипотезу об отсутствии разницы между группами. На заданном уровне надежности не найдено статистически значимое различие между группами.")

- Delta t-test:
Значение T статистики:  -1.0902035199525153
p-value: 0.2756235
Невозможно опровергнуть гипотезу об отсутствии разницы между группами. На заданном уровне надежности не найдено статистически значимое различие между группами.


## 3.3. CUPED- метод<a class="anchor" id="h3.3"></a>
[<font size="2">(к содержанию)</font>](#to_content1.)

Предпосылки:
1. Большая дисперсия в данных (подходит и для количественных метрик)
2. Нужны данные до теста

In [23]:
import numpy as np
import pandas as pd
from collections import namedtuple
import scipy.stats as sps
ExperimentComparisonResults = namedtuple('ExperimentComparisonResults', 
                                        ['pvalue', 'effect', 'ci_length', 'left_bound', 'right_bound'])

In [24]:
n = 1000 #размер выборки
p_val_cuped = 0.05 #уровень надежности

control = np.random.normal(90, 20, n)
test = np.random.normal(90, 15, n)
control_before = np.random.normal(90, 20, n)
test_before = np.random.normal(90, 17, n)

## импорт своих значений
# control = pd.read_csv()
# test = pd.read_csv()
# control_before = pd.read_csv()
# test_before = pd.read_csv()

In [25]:
# T-test
def absolute_ttest(control, test):
    mean_control = np.mean(control)
    mean_test = np.mean(test)
    var_mean_control  = np.var(control) / len(control)
    var_mean_test  = np.var(test) / len(test)
    
    difference_mean = mean_test - mean_control
    difference_mean_var = var_mean_control + var_mean_test
    difference_distribution = sps.norm(loc=difference_mean, scale=np.sqrt(difference_mean_var))

    left_bound, right_bound = difference_distribution.ppf([p_val_cuped/2, 1-p_val_cuped/2])
    ci_length = (right_bound - left_bound)
    pvalue = 2 * min(difference_distribution.cdf(0), difference_distribution.sf(0))
    effect = difference_mean
    return ExperimentComparisonResults(pvalue, effect, ci_length, left_bound, right_bound)

In [26]:
# Вводим фунцию для расчет критерия тета
def cuped_ttest(control, test, control_before, test_before):
    theta = (np.cov(control, control_before)[0, 1] + np.cov(test, test_before)[0, 1]) /\
                (np.var(control_before) + np.var(test_before))
    control_cup = control - theta * control_before
    test_cup = test - theta * test_before
    return absolute_ttest(control_cup, test_cup)

In [27]:
# Запускаю критерий
cuped_ci = cuped_ttest(control, test, control_before, test_before)
ttest_ci = absolute_ttest(control, test)
p_val_cuped_test = cuped_ci[0]

In [28]:
print("- CUPED t-test:")
# print("Значение T статистики: ", t_val)
print(f"p-value: {p_val_cuped_test:0.7f}")

# Проверяем есть ли изменение
if p_val_cuped_test < p_val_cuped:
    print("Невозможно подтвердить гипотезу об отсутствии разницы между группами. Найденое различие на заданном уровне надежности может наблюдаться")
else:
    print("Невозможно опровергнуть гипотезу об отсутствии разницы между группами. На заданном уровне надежности не найдено статистически значимое различие между группами.")

- CUPED t-test:
p-value: 0.6573714
Невозможно опровергнуть гипотезу об отсутствии разницы между группами. На заданном уровне надежности не найдено статистически значимое различие между группами.


# 4. Множественное тестирование<a class="anchor" id="h4"></a>

## 4.1. Поправка Бонферонни <a class="anchor" id="h4.1"></a>
[<font size="2">(к содержанию)</font>](#to_content1.)

Предпосылки:
1. сравнение не больше 3 вариантов между собой

Обычный t-test  с уровнем надежности alpha/n, где n - кол-во вариантов

In [29]:
import scipy
from scipy import stats
import pandas as pd
import numpy as np

In [30]:
k = 3 #кол-во вариантов
n = 1000 #размер выборки
p_val_n = 0.05/k #уровень надежности

bonf_a = np.random.normal(90, 20, n)
bonf_b = np.random.normal(90, 15, n)
bonf_c = np.random.normal(89, 20, n)

## импорт своих значений
# bonf_a = pd.read_csv()
# bonf_b = pd.read_csv()
# bonf_c = pd.read_csv()

In [31]:
n_a = len(bonf_a)
n_b = len(bonf_b)
n_c = len(bonf_c)

s_a = np.sqrt(np.var(bonf_a, ddof=1))
s_b = np.sqrt(np.var(bonf_b, ddof=1))
s_c = np.sqrt(np.var(bonf_c, ddof=1))

s_p_ab = np.sqrt(((n_a-1) * s_a**2 +
               (n_b-1) * s_b**2)
              / (n_a + n_b - 2))

s_p_ac = np.sqrt(((n_a-1) * s_a**2 +
               (n_c-1) * s_c**2)
              / (n_a + n_c - 2))

s_p_bc = np.sqrt(((n_b-1) * s_b**2 +
               (n_c-1) * s_c**2)
              / (n_b + n_c - 2))

In [32]:
# t value
t_val_ab = (bonf_a.mean() - bonf_b.mean()) / (s_p_ab * np.sqrt(1/n_a + 1/n_b))
t_val_ac = (bonf_a.mean() - bonf_c.mean()) / (s_p_ac * np.sqrt(1/n_a + 1/n_c))
t_val_bc = (bonf_b.mean() - bonf_c.mean()) / (s_p_bc * np.sqrt(1/n_b + 1/n_c))

# pvalue
p_val_ab_  = scipy.stats.t.sf(abs(t_val_ab), df=2*n-2)*2
p_val_ac_  = scipy.stats.t.sf(abs(t_val_ac), df=2*n-2)*2
p_val_bc_  = scipy.stats.t.sf(abs(t_val_bc), df=2*n-2)*2


In [33]:
print("- T-test для групп AB:")
print("Значение T статистики: ", t_val_ab)
print(f"p-value: {p_val_ab_:0.7f}")

# Проверяем есть ли изменение
if p_val_ab_ < p_val_n:
    print("Невозможно подтвердить гипотезу об отсутствии разницы между группами. Найденое различие на заданном уровне надежности может наблюдаться")
else:
    print("Невозможно опровергнуть гипотезу об отсутствии разницы между группами. На заданном уровне надежности не найдено статистически значимое различие между группами.")

- T-test для групп AB:
Значение T статистики:  0.9343198251295213
p-value: 0.3502518
Невозможно опровергнуть гипотезу об отсутствии разницы между группами. На заданном уровне надежности не найдено статистически значимое различие между группами.


In [34]:
print("T-test для групп BC:")
print("Значение T статистики: ", t_val_bc)
print(f"p-value: {p_val_bc_:0.7f}")

# Проверяем есть ли изменение
if p_val_bc_ < p_val_n:
    print("Невозможно подтвердить гипотезу об отсутствии разницы между группами. Найденое различие на заданном уровне надежности может наблюдаться")
else:
    print("Невозможно опровергнуть гипотезу об отсутствии разницы между группами. На заданном уровне надежности не найдено статистически значимое различие между группами.")

T-test для групп BC:
Значение T статистики:  0.3786720714632882
p-value: 0.7049716
Невозможно опровергнуть гипотезу об отсутствии разницы между группами. На заданном уровне надежности не найдено статистически значимое различие между группами.


In [35]:
print("T-test для групп AC:")
print("Значение T статистики: ", t_val_ac)
print(f"p-value: {p_val_ac_:0.7f}")

# Проверяем есть ли изменение
if p_val_ac_ < p_val_n:
    print("Невозможно подтвердить гипотезу об отсутствии разницы между группами. Найденое различие на заданном уровне надежности может наблюдаться")
else:
    print("Невозможно опровергнуть гипотезу об отсутствии разницы между группами. На заданном уровне надежности не найдено статистически значимое различие между группами.")

T-test для групп AC:
Значение T статистики:  1.164810584970887
p-value: 0.2442347
Невозможно опровергнуть гипотезу об отсутствии разницы между группами. На заданном уровне надежности не найдено статистически значимое различие между группами.


## 4.2. Метод Холма <a class="anchor" id="h4.2"></a>
[<font size="2">(к содержанию)</font>](#to_content1.)

Предпосылки:
1. Если вариантов больше трех


In [36]:
import scipy
from scipy import stats
import pandas as pd
import numpy as np
from statsmodels.sandbox.stats.multicomp import multipletests

In [37]:
k = 3 #кол-во вариантов
n = 1000 #размер выборки
p_val_holm = 0.05 #уровень надежности

holm_a = np.random.normal(90, 15, n)
holm_b = np.random.normal(90, 15, n)
holm_c = np.random.normal(90, 15, n)

## импорт своих значений
# holm_a = pd.read_csv()
# holm_b = pd.read_csv()
# holm_c = pd.read_csv()

In [38]:
stat_ab, p_ab = stats.ttest_ind(holm_a, holm_b)

stat_bc, p_bc = stats.ttest_ind(holm_b, holm_c)

stat_ac, p_ac = stats.ttest_ind(holm_a, holm_c)

In [39]:
print("Изначальные p-value: ",sorted([p_ab, p_bc, p_ac]))


Изначальные p-value:  [0.45931736498816456, 0.5799048541477703, 0.8456747280477008]


In [40]:
h0_result, p_val_corrected, p_val, p_val_new = multipletests([p_ab, p_bc, p_ac], alpha=p_val_holm, 
                     method='holm', is_sorted = True)

In [41]:
print("- Holm's test result:")

# Тестируются гипотезы p_val_corrected < p_val_holm
print("Результаты проверки гипотезы: ", h0_result)
print("Скорректированные p_value: ", p_val_corrected)

# Проверяем есть ли изменение
for i in p_val_corrected:
    if i < p_val_holm:
        print("Невозможно подтвердить гипотезу об отсутствии разницы между группами. Найденое различие на заданном уровне надежности может наблюдаться")
    else:
        print("Невозможно опровергнуть гипотезу об отсутствии разницы между группами. На заданном уровне надежности не найдено статистически значимое различие между группами.")

- Holm's test result:
Результаты проверки гипотезы:  [False False False]
Скорректированные p_value:  [1. 1. 1.]
Невозможно опровергнуть гипотезу об отсутствии разницы между группами. На заданном уровне надежности не найдено статистически значимое различие между группами.
Невозможно опровергнуть гипотезу об отсутствии разницы между группами. На заданном уровне надежности не найдено статистически значимое различие между группами.
Невозможно опровергнуть гипотезу об отсутствии разницы между группами. На заданном уровне надежности не найдено статистически значимое различие между группами.


## 4.2. Метод Бенджамини — Хохберга <a class="anchor" id="h4.3"></a>
[<font size="2">(к содержанию)</font>](#to_content1.)

Предпосылки:
1. Если вариантов больше трех



In [42]:
import scipy
from scipy import stats
import pandas as pd
import numpy as np
from statsmodels.sandbox.stats.multicomp import multipletests

In [43]:
k = 3 #кол-во вариантов
n = 1000 #размер выборки
p_val_benj_hohberg = 0.05 #уровень надежности

benj_hohberg_a = np.random.normal(90, 20, n)
benj_hohberg_b = np.random.normal(90, 15, n)
benj_hohberg_c = np.random.normal(89, 20, n)

## импорт своих значений
# benj_hohberg_a = pd.read_csv()
# benj_hohberg_b = pd.read_csv()
# benj_hohberg_c = pd.read_csv()

In [44]:
stat_ab_bh, p_ab_bh = stats.ttest_ind(benj_hohberg_a, benj_hohberg_b)

stat_bc_bh, p_bc_bh = stats.ttest_ind(benj_hohberg_b, benj_hohberg_c)

stat_ac_bh, p_ac_bh = stats.ttest_ind(benj_hohberg_a, benj_hohberg_c)

In [45]:
print("Изначальные p-value: ",sorted([p_ab_bh, p_bc_bh, p_ac_bh]))

Изначальные p-value:  [0.8998749060134313, 0.9007387730504447, 0.9974684703865129]


In [46]:
h0_result_bh, p_val_corrected_bh, p_val_bh, p_val_new_bh = multipletests([p_ab, p_bc, p_ac], alpha=p_val_benj_hohberg, 
                     method='fdr_bh', is_sorted = False)

In [47]:
print("- BHM's test result:")
# Тестируются гипотезы p_val_corrected < p_val_holm
print("Результаты проверки гипотезы: ", h0_result_bh)
print("Скорректированные p_value: ", p_val_corrected_bh)


# Проверяем есть ли изменение
for i in p_val_corrected_bh:
    if i < p_val_benj_hohberg:
        print("Невозможно подтвердить гипотезу об отсутствии разницы между группами. Найденое различие на заданном уровне надежности может наблюдаться")
    else:
        print("Невозможно опровергнуть гипотезу об отсутствии разницы между группами. На заданном уровне надежности не найдено статистически значимое различие между группами.")

- BHM's test result:
Результаты проверки гипотезы:  [False False False]
Скорректированные p_value:  [0.84567473 0.84567473 0.84567473]
Невозможно опровергнуть гипотезу об отсутствии разницы между группами. На заданном уровне надежности не найдено статистически значимое различие между группами.
Невозможно опровергнуть гипотезу об отсутствии разницы между группами. На заданном уровне надежности не найдено статистически значимое различие между группами.
Невозможно опровергнуть гипотезу об отсутствии разницы между группами. На заданном уровне надежности не найдено статистически значимое различие между группами.
